# Stochastic compute GCM fluxes
Clone of the GCM (MiMA) compute fluxes, however now using a stochastic implementation.

Care needs to be taken in this case to isolate blocks.

In [1]:
### Master Variables
dc = 0.4 # Spectral resolution
exclude_unbroken = False # Exclude waves that are unbroken at top level from the momentum flux calculations? 
use_intrinsic_c = 'always' # always center distributon on zero intrinsic phase speed vs never (center on zero ground relative speed) or switch at latitude ? 
Fs0 = 4.3e-3 # Pa, initial momentum flux at source level
cw = 35 # m/s, gravity wave phase speed source distribution half width at half maximum 
expname=f'{use_intrinsic_c}_{"exclude" if exclude_unbroken else "include"}_unbroken_1year'
expname

'always_include_unbroken_1year'

## dask/SLURM setup

In [2]:
from dask_jobqueue import SLURMCluster
NCORES = 8
NPROCESS = 8
NCORESPERPROCESS = NCORES//NPROCESS
constraints = ['-C \"CLASS:SH4_CBASE|CLASS:SH4_CPERF\"'] # SH4 nodes are the fastest, and mixing node gens seems to cause ib0 issues.
cluster = SLURMCluster(queue='serc',memory='96GiB',cores=NCORES,processes=NPROCESS,walltime='06:00:00',job_extra_directives=constraints,log_directory='/scratch/users/robcking/dask_worker_logs')
cluster.scale(jobs=25) # roughly but tune to scheduler 
cluster

Dashboard: http://10.20.3.15:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.20.3.15:38031,Workers: 0
Dashboard: http://10.20.3.15:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
from dask.distributed import Client 
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.20.3.15:8787/status,
Dashboard: http://10.20.3.15:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.20.3.15:38031,Workers: 0
Dashboard: http://10.20.3.15:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Load Data

In [6]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from ad99py.ad99 import AlexanderDunkerton1999
from ad99py.masks import mask_dataset,load_mask
from ad99py.constants import GRAV,R_DRY,C_P


DATA_PATH = 'data' # Load in raw data from GCM, here we use MiMA that has been interpolated to the ERA5 pressure levels. 
ds = xr.open_dataset(os.path.join(DATA_PATH,'atmos_4xdaily_interp_1year.nc'),chunks={'time':16})
ds = ds[['ucomp','temp','vcomp','height']]
ds = ds.isel(level=slice(None,None,-1))
ds = ds.rename(lat='latitude',lon='longitude')
ds = ds.transpose('time','longitude','latitude','level')
ds = ds.chunk({'time':16})
ds

<xarray.Dataset> Size: 7GB
Dimensions:    (time: 1440, level: 37, latitude: 64, longitude: 128)
Coordinates:
  * longitude  (longitude) float32 512B 0.0 2.812 5.625 ... 351.6 354.4 357.2
  * latitude   (latitude) float32 256B -87.86 -85.1 -82.31 ... 82.31 85.1 87.86
  * level      (level) float32 148B 1e+03 975.0 950.0 925.0 ... 5.0 3.0 2.0 1.0
  * time       (time) object 12kB 0025-01-01 00:00:00 ... 0025-12-30 18:00:00
Data variables:
    ucomp      (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(16, 128, 64, 37), meta=np.ndarray>
    temp       (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(16, 128, 64, 37), meta=np.ndarray>
    vcomp      (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(16, 128, 64, 37), meta=np.ndarray>
    height     (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(16, 128, 64, 37), meta=np.ndarray>
Attributes:
    filename:     atmos_4xdaily_interp.nc
    title:        MiMA
    history:      Tue Jul 08 18:56:01 2025: plevel.sh -a -i atmos_4xdaily_fla...
    NCO:          netCDF Operators version 5.2.4 (Homepage = http://nco.sf.ne...
    fre-nctools:  FRE NCTools version 2024.05 (git: 2024.05.01-3-gffa6ce6)

Load Data

In [ ]:
us = ds.ucomp.data
vs = ds.vcomp.data
zs = ds.height.data
temps = ds.temp.data
level = ds.level.data

In [ ]:
import dask.array as da
from ad99py.variables import bouyancy_freq_squared,density

Ns = bouyancy_freq_squared(temps,zs)**0.5
rho = density(temps,level)
lat=da.broadcast_to(ds.latitude.data[None,None,:].data,us.shape[:-1])[...,None].rechunk((16,-1,-1,-1))


## Define Blocks

In [ ]:
SEED = 42 # for reproducibility
master = np.random.SeedSequence(SEED)
n_blocks = us.shape[0]
seeds = master.spawn(n_blocks)
seeds